In [13]:
import os
import json
import pandas as pd
import traceback

In [14]:
from langchain.chat_models import ChatOpenAI

In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
KEY = os.getenv("OPENAI_API_KEY")

In [19]:
client = ChatOpenAI(openai_api_key = KEY, model = "gpt-3.5-turbo",temperature = 0.6)

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
data = PyPDFLoader("./javase8.pdf")

In [ ]:
page = data.load_and_split()

In [ ]:
# 48-55
req_data = page[47:55]

In [ ]:
format = {
   "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [ ]:
generate_template = """
using the below data generate me {number_of_questions} with a {level} difficulty level.
{data} generate me output in the below format 
{format}
"""

In [ ]:
generate_template

'\nusing the below data generate me {number_of_questions} with a {level} difficulty level.\n{data} generate me output in the below format \n{format}\n'

In [ ]:
generate_mcq = PromptTemplate(
    input_variables=["data","number_of_questions","level","format"],
    template= generate_template
)

In [ ]:
# generate_mcq = PromptTemplate.from_template("using the below data generate me {number_of_questions} with a {level} difficulty level.{data} generate me output in the below format {format}")

In [ ]:
generate_mcq_prompt = LLMChain(llm = client, prompt=generate_mcq, output_key="base_mcq")

In [ ]:
validate_template = """
validate these mcqs {base_mcq} are they correct. if there is any mistake modify them as required.
"""

In [ ]:
validate_mcq = PromptTemplate(
    input_variables=["base_mcq"],
    template= validate_template
)

In [ ]:
validate_mcq_prompt = LLMChain(llm = client, prompt= validate_mcq, output_key="final_mcq")

In [ ]:
mcqs = SequentialChain(chains= [generate_mcq_prompt, validate_mcq_prompt],input_variables=["data","number_of_questions","level","format"],output_variables=["base_mcq","final_mcq"])

In [ ]:
response = mcqs({"data" : req_data, "number_of_questions" : "5", "level" : "Easy" , "format" : format})

In [ ]:
response

{'data': [Document(page_content='56 CHAPTER 3: DECLARATIONS\nOverloaded Constructors\nLike methods, constructors can be overloaded. Since the constructors in a class all\nhave the same name as the class, their si gnatures are differenti ated by their param-\neter lists. In the following example, the class Light  now provides explicit imple-\nmentation of the no-argument constructor at (1) and that of a non-zero argument\nconstructor at (2). The constructors are overloaded, as is evident by their signa-\ntures. The non-zero argument constructor at (2) is called when an object of the class\nLight  is created at (3), and the no-argument constructor is likewise called at (4).\nOverloading of constructors allows appropriate initialization of objects on\ncreation, depending on the constructor invoked (see chaining of constructors in\n§7.5, p. 282). It is recommended to use the @param  tag in a Javadoc comment to doc-\nument the formal parameters of a constructor. \nclass Light {\n  // ...\n 

In [ ]:
print(response["final_mcq"])

1. Which one of these declarations is a valid method declaration?
a. void method1 { /* ... */ }
b. void method2() { /* ... */ }
c. void method3(void) { /* ... */ }
d. method4() { /* ... */ }
e. method5(void) { /* ... */ }
Correct answer: b

2. Which statements, when inserted at (1), will not result in compile-time errors?
a. i = this.planets;
b. i = this.suns;
c. this = new ThisUsage();
d. this.i = 4;
e. this.suns = planets;
Correct answer: a, b

3. Given the following pairs of method declarations, which statements are true?
a. The first pair of methods will compile, and overload the method name fly.
b. The second pair of methods will compile, and overload the method name fall.
c. The third pair of methods will compile, and overload the method name glide.
d. The first pair of methods will not compile.
e. The second pair of methods will not compile.
f. The third pair of methods will not compile.
Correct answer: a, b

4. Given a class named Book, which one of these constructor declaratio